In [1]:
import os
import json
import requests
import pandas as pd
import numpy as np
import datetime

* Base url for api call for each user.

In [2]:
# give information about user's submission history
URL1 = "https://codeforces.com/api/user.status?handle="
# give informaiton about user's competition history
URL2 = "https://codeforces.com/api/user.rating?handle="

* Helper function to retrive data from server

In [3]:
def getSubData(uname):
    return json.loads(requests.get(URL1 + uname).content)['result']

def getCompiData(uname):
    return json.loads(requests.get(URL2 + uname).content)['result']

In [27]:
user1 = 'poojanpatel101'
subData1 = getSubData(user1)
compiData1 = getCompiData(user1)

In [28]:
print("Total Number of sumbission for user %s is %d" % (user,len(subData)))

Total Number of sumbission for user avid_avi is 411


In [29]:
print("Total Number of compitition participated for user %s is %d" % (user,len(compiData)))

Total Number of compitition participated for user avid_avi is 28


# Plot 1:
* This plot shows comparison between two users compititon rating.
* It will allow user to track and compare their progress with other user from time to time.
* Each user starts with 1500 rating.
* This will allow one team member compare their rating and improve accordinly.

In [31]:
import plotly.plotly as py
import plotly.graph_objs as go

In [114]:
def find_rating(date, data):
    for d in data:
        if date == datetime.datetime.fromtimestamp(float(d['ratingUpdateTimeSeconds'])):
            return d['newRating']

def plot_two_rating(username1,username2):
    compiData1 = getCompiData(username1)
    compiData2 = getCompiData(username2)
    
    date_user1 = sorted(list(set([datetime.datetime.fromtimestamp(float(i['ratingUpdateTimeSeconds'])) for i in compiData1])))
    date_user2 = sorted(list(set([datetime.datetime.fromtimestamp(float(i['ratingUpdateTimeSeconds'])) for i in compiData2])))

    total_dates = sorted(list(set(date_user1 + date_user2)))
    
    data1 = []
    data2 = []
    for date_ in total_dates:
        if date_ in date_user1:
            data1.append((date_, find_rating(date_,compiData1)))
        else:
            if len(data1)>0:
                data1.append((date_,data1[-1][1]))
            else:
                data1.append((date_,1500))

        if date_ in date_user2:
            data2.append((date_, find_rating(date_,compiData2)))
        else:
            if len(data2)>0:
                data2.append((date_,data2[-1][1]))
            else:
                data2.append((date_,1500))

    pldata1 = go.Scatter(x=[x[0] for x in data1], y=[x[1] for x in data1])
    pldata2 = go.Scatter(x=[x[0] for x in data2], y=[x[1] for x in data2])
    
    fig = dict(data=[pldata1,pldata2])
    return py.iplot(fig, filename = "Manually Set Range")

In [117]:
plot_two_rating('tourist','petr')